In [ ]:
# IMPORT ALL YOUR LIBRARIES
!pip install transformers
!pip install datasets
!pip install accelerate >=0.21.0
!pip install transformers[torch]

import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
# from datasets import Dataset
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score
from transformers import Trainer, TrainingArguments
import accelerate

import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
# from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/HammadxSaj/SemEval_Task11_Dataset/refs/heads/main/train_english.csv"
!wget -P /kaggle/working -nc "https://raw.githubusercontent.com/HammadxSaj/SemEval_Task11_Dataset/refs/heads/main/test_eng.csv"

--2024-11-29 03:50:57--  https://raw.githubusercontent.com/HammadxSaj/SemEval_Task11_Dataset/refs/heads/main/train_english.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 316456 (309K) [text/plain]
Saving to: ‘/kaggle/working/train_english.csv’

train_english.csv   100%[===================>] 309.04K  --.-KB/s    in 0.004s  

2024-11-29 03:50:57 (77.9 MB/s) - ‘/kaggle/working/train_english.csv’ saved [316456/316456]

--2024-11-29 03:50:57--  https://raw.githubusercontent.com/HammadxSaj/SemEval_Task11_Dataset/refs/heads/main/test_eng.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connecte

In [ ]:
df = pd.read_csv('/kaggle/working/train_english.csv')
df

id  \
0     eng_train_track_a_00001   
1     eng_train_track_a_00002   
2     eng_train_track_a_00003   
3     eng_train_track_a_00004   
4     eng_train_track_a_00005   
...                       ...   
2763  eng_train_track_a_02764   
2764  eng_train_track_a_02765   
2765  eng_train_track_a_02766   
2766  eng_train_track_a_02767   
2767  eng_train_track_a_02768   

                                                   text  Anger  Fear  Joy  \
0                                   But not very happy.      0     0    1   
1     Well she's not gon na last the whole song like...      0     0    1   
2     She sat at her Papa's recliner sofa only to mo...      0     0    0   
3                       Yes, the Oklahoma city bombing.      1     1    0   
4                          They were dancing to Bolero.      0     0    1   
...                                                 ...    ...   ...  ...   
2763                 "Yeah, but did you just find that?      0     1    0   
2764  I did as little as possible with my right hand...      0     0    0   
2765                            Okay that sucks, right?      1     0    0   
2766  The spark leaped through his body into mine, a...      0     1    0   
2767  He had 4 inches and 40 pounds on me and I stil...      0     0    1   

      Sadness  Surprise  
0           1         0  
1           0         0  
2           0         0  
3           1         1  
4           0         0  
...       ...       ...  
2763        0         1  
2764        0         0  
2765        1         0  
2766        0         1  
2767        0         1  

[2768 rows x 7 columns]

In [ ]:
df_test = pd.read_csv('/kaggle/working/test_eng.csv')
df_test

id                                               text  \
0    eng_dev_track_a_00001            My mouth fell open `` No, no, no... I..   
1    eng_dev_track_a_00002  You can barely make out your daughter's pale f...   
2    eng_dev_track_a_00003  But after blinking my eyes for a few times lep...   
3    eng_dev_track_a_00004  Slowly rising to my feet I came to the conclus...   
4    eng_dev_track_a_00005  I noticed this months after moving in and doin...   
..                     ...                                                ...   
111  eng_dev_track_a_00112                       "ARcH stop your progression.   
112  eng_dev_track_a_00113        This 'star', starts to move across the sky.   
113  eng_dev_track_a_00114                                  and my feet hurt.   
114  eng_dev_track_a_00115        so i cried my eyes out and did the drawing.   
115  eng_dev_track_a_00116                              They were coal black.   

     Anger  Fear  Joy  Sadness  Surprise  
0      NaN   NaN  NaN      NaN       NaN  
1      NaN   NaN  NaN      NaN       NaN  
2      NaN   NaN  NaN      NaN       NaN  
3      NaN   NaN  NaN      NaN       NaN  
4      NaN   NaN  NaN      NaN       NaN  
..     ...   ...  ...      ...       ...  
111    NaN   NaN  NaN      NaN       NaN  
112    NaN   NaN  NaN      NaN       NaN  
113    NaN   NaN  NaN      NaN       NaN  
114    NaN   NaN  NaN      NaN       NaN  
115    NaN   NaN  NaN      NaN       NaN  

[116 rows x 7 columns]

In [ ]:
from bs4 import BeautifulSoup
import re

# Step 2: Clean the text data
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove numbers (but keep special characters)
    text = re.sub(r'[^a-zA-Z\s!\'"?,.;:]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

# Apply the clean_text function
df_test['text'] = df_test['text'].apply(clean_text)

# Step 3-5: (Same as your original code)
df_test = df_test[df_test['text'].notna() & (df_test['text'] != '')]

# Ensure emotion columns are integers
# emotion_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
emotion_columns = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
df_test[emotion_columns] = df_test[emotion_columns]

# Save the processed data back to CSV
output_path = '/kaggle/working/cleaned_test_english.csv'
df_test.to_csv(output_path, index=False)

print(f"Data preprocessing completed. File 'cleaned_test_english.csv' has been created in the /kaggle/working directory.")

# Display statistics and processed examples
print(f"Total samples: {len(df_test)}")
print("\nEmotion distribution in the dataset:")
print(df_test[emotion_columns].sum())
print("\nFirst few processed examples:")
print(df_test.head())


Data preprocessing completed. File 'cleaned_test_english.csv' has been created in the /kaggle/working directory.
Total samples: 116

Emotion distribution in the dataset:
Anger       0.0
Fear        0.0
Joy         0.0
Sadness     0.0
Surprise    0.0
dtype: float64

First few processed examples:
                      id                                               text  \
0  eng_dev_track_a_00001               my mouth fell open no, no, no... i..   
1  eng_dev_track_a_00002  you can barely make out your daughter's pale f...   
2  eng_dev_track_a_00003  but after blinking my eyes for a few times lep...   
3  eng_dev_track_a_00004  slowly rising to my feet i came to the conclus...   
4  eng_dev_track_a_00005  i noticed this months after moving in and doin...   

   Anger  Fear  Joy  Sadness  Surprise  
0    NaN   NaN  NaN      NaN       NaN  
1    NaN   NaN  NaN      NaN       NaN  
2    NaN   NaN  NaN      NaN       NaN  
3    NaN   NaN  NaN      NaN       NaN  
4    NaN   NaN  NaN     

<ipython-input-5-d1ccaa509ba7>:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [ ]:
if 'id' in df.columns:
    df = df.drop(['id'], axis=1)
from bs4 import BeautifulSoup
import re

# Step 2: Clean the text data
def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    # Remove numbers (but keep special characters)
    text = re.sub(r'[^a-zA-Z\s!\'"?,.;:]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

# df['text'] = df['text'].apply(clean_text)

# Step 3: Remove rows with empty text
df = df[df['text'].notna() & (df['text'] != '')]

# Step 4: Ensure emotion columns are integers
#emotion_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
emotion_columns = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
df[emotion_columns] = df[emotion_columns].astype(int)

# Step 5: Save the processed data back to CSV
output_path = '/kaggle/working/cleaned_train_english.csv'
df.to_csv(output_path, index=False)

print(f"Data preprocessing completed. File 'cleaned_train_english.csv' has been created in the /kaggle/working directory.")

# Optional: Display some statistics about the data
print(f"Total samples: {len(df)}")
print("\nEmotion distribution in the dataset:")
print(df[emotion_columns].sum())

# Optional: Display a few processed examples
print("\nFirst few processed examples:")
print(df.head())

Data preprocessing completed. File 'cleaned_train_english.csv' has been created in the /kaggle/working directory.
Total samples: 2768

Emotion distribution in the dataset:
Anger        333
Fear        1611
Joy          674
Sadness      878
Surprise     839
dtype: int64

First few processed examples:
                                                text  Anger  Fear  Joy  \
0                                But not very happy.      0     0    1   
1  Well she's not gon na last the whole song like...      0     0    1   
2  She sat at her Papa's recliner sofa only to mo...      0     0    0   
3                    Yes, the Oklahoma city bombing.      1     1    0   
4                       They were dancing to Bolero.      0     0    1   

   Sadness  Surprise  
0        1         0  
1        0         0  
2        0         0  
3        1         1  
4        0         0  


In [ ]:
from transformers import BertTokenizer, BertModel
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5, problem_type="multi_label_classification")
# model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=6, problem_type="multi_label_classification")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import load_dataset, Dataset, DatasetDict

In [ ]:
train_dataset = Dataset.from_pandas(df)

train_dataset

dataset = train_dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
val_dataset = dataset['test']
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'],
        num_rows: 2214
    })
    test: Dataset({
        features: ['text', 'Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'],
        num_rows: 554
    })
})


In [ ]:
# Convert train_dataset and val_dataset back to Pandas DataFrame
train_df = train_dataset.to_pandas()
val_df = val_dataset.to_pandas()

# Save to CSV
train_df.to_csv('train_dataset.csv', index=False)
val_df.to_csv('val_dataset.csv', index=False)

print("CSV files have been saved.")


CSV files have been saved.


In [ ]:
train_df.head()

text  Anger  Fear  Joy  \
0  Grant you I still don't have total use of my s...      0     0    1   
1  Now I am resting and I have to do my ear drops...      0     1    0   
2          The stone inside my heart was removed; ).      0     0    1   
3                   But it's a very strange feeling.      0     1    0   
4  Edward rid me of my distraction by biting my s...      0     1    0   

   Sadness  Surprise  
0        0         0  
1        0         0  
2        0         0  
3        0         1  
4        0         0

In [ ]:
val_df.head()

text  Anger  Fear  Joy  \
0  & # x 26; nbsp; Walking back in the sun was no...      0     1    0   
1  With an infant, stroller and backup, I had my ...      0     1    0   
2  I always had my nose in a book when I was youn...      0     0    0   
3  3.Woke up from a horrible dream of my ex boyfr...      1     1    0   
4  Any potential bride is warned:  The groomsmen ...      0     1    0   

   Sadness  Surprise  
0        0         0  
1        0         0  
2        1         0  
3        1         0  
4        0         1

In [ ]:
cols = train_df.columns[1:]
cols

Index(['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise'], dtype='object')

In [ ]:
training_labels = train_df[cols]
training_labels = training_labels.to_numpy()
training_labels = training_labels.astype(np.float32)

train_sentences = train_df.text.values.tolist()

validation_labels = val_df[cols]
validation_labels = validation_labels.to_numpy()
validation_labels = validation_labels.astype(np.float32)

validation_sentences = val_df.text.values.tolist()

In [ ]:
# training_sentence_encoding = tokenizer(train_sentences, padding = True, truncation = True, max_length = 512)
# validation_sentence_encoding = tokenizer(validation_sentences, padding = True, truncation = True, max_length = 512)

train_encodings = tokenizer(train_sentences, padding=True, truncation=True, max_length=512)
val_encodings = tokenizer(validation_sentences, padding=True, truncation=True, max_length=512)

In [ ]:
from datasets import Dataset
import torch
import numpy as np

train_dataset_new = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': training_labels
})

validation_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': validation_labels
})

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)

epochs = 4
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(df) * epochs)

training_arguments = TrainingArguments(
    output_dir="./outputs",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    save_strategy="epoch"
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import wandb

# Initialize wandb with mode disabled
wandb.init(mode="disabled")

In [ ]:
#defining the metrics to be calculated during training
def multi_label_metrics(predictions, labels,):
    sigmoid = torch.nn.Sigmoid()
    predictions = sigmoid(torch.tensor(predictions))
    y_predictions = np.zeros(predictions.shape)
    y_true = labels
    y_predictions[np.where(predictions >= 0.4)] = 1
    accuracy = accuracy_score(y_true, y_predictions)
    f1 = f1_score(y_true, y_predictions, average='micro')
    auc = roc_auc_score(y_true, y_predictions, average='micro')

    metrics = {
        'accuracy': accuracy,
        'f1': f1,
        'auc': auc
    }

    return metrics

#defining the function to compute the metrics for each batch of data
def compute_metrics(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(preds, p.label_ids)

    return result

#defining the trainer
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset_new,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


TrainOutput(global_step=556, training_loss=0.27737036111543506, metrics={'train_runtime': 258.5485, 'train_samples_per_second': 34.253, 'train_steps_per_second': 2.15, 'total_flos': 559787959313136.0, 'train_loss': 0.27737036111543506, 'epoch': 4.0})

In [ ]:
#Saving the model
model.save_pretrained("/content/")

In [ ]:
#checking the training history of the model
trainer.state.log_history

[{'eval_loss': 0.23200225830078125,
  'eval_accuracy': 0.6616666666666666,
  'eval_f1': 0.6615720524017468,
  'eval_auc': 0.784774193548387,
  'eval_runtime': 4.3912,
  'eval_samples_per_second': 136.638,
  'eval_steps_per_second': 8.654,
  'epoch': 1.0,
  'step': 150},
 {'eval_loss': 0.17762213945388794,
  'eval_accuracy': 0.7633333333333333,
  'eval_f1': 0.7763023493360572,
  'eval_auc': 0.8640322580645161,
  'eval_runtime': 4.3838,
  'eval_samples_per_second': 136.868,
  'eval_steps_per_second': 8.668,
  'epoch': 2.0,
  'step': 300},
 {'eval_loss': 0.18062974512577057,
  'eval_accuracy': 0.7483333333333333,
  'eval_f1': 0.7611336032388664,
  'eval_auc': 0.8579354838709677,
  'eval_runtime': 4.3023,
  'eval_samples_per_second': 139.459,
  'eval_steps_per_second': 8.832,
  'epoch': 3.0,
  'step': 450},
 {'loss': 0.2038,
  'grad_norm': 0.8803262114524841,
  'learning_rate': 8.333333333333334e-06,
  'epoch': 3.3333333333333335,
  'step': 500},
 {'eval_loss': 0.17588746547698975,
  'eval

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the model and tokenizer
model = BertForSequenceClassification.from_pretrained("/content/")
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Make sure to set the model to evaluation mode
model.eval()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
# Test data (assuming you have a DataFrame with a column 'text')
test_sentences = df_test['text'].tolist()

# Tokenize the test data
test_encodings = tokenizer(test_sentences, padding=True, truncation=True, max_length=512, return_tensors="pt")

In [ ]:
# Move the encodings to the same device as the model (GPU if available, otherwise CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Pass the test data through the model to get predictions
with torch.no_grad():
    input_ids = test_encodings['input_ids'].to(device)
    attention_mask = test_encodings['attention_mask'].to(device)
    outputs = model(input_ids, attention_mask=attention_mask)

# Apply sigmoid to get probabilities
sigmoid = torch.nn.Sigmoid()
predictions = sigmoid(outputs.logits)

# Convert probabilities to binary predictions (assuming a threshold of 0.5)
predicted_labels = (predictions > 0.5).int()

# Convert predictions to a more readable format (back to CPU if necessary)
predicted_labels = predicted_labels.cpu().numpy()


In [ ]:
# Assuming your labels are in the order ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
label_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']

# Convert predictions to a DataFrame for easy viewing
predicted_df = pd.DataFrame(predicted_labels, columns=label_columns)

# Combine with the original test sentences
result_df = pd.concat([df_test['text'], predicted_df], axis=1)

# Display the result
print(result_df.head())


                                                text  Anger  Disgust  Fear  \
0  Auf die Frage an Präsident Biden, ob die Besch...      0        0     0   
1  Sind Organe von adipösen Menschen überhaupt "b...      0        0     0   
2  Die Kriegsbegeisterung kennt anscheinend keine...      0        0     0   
3                      Selenskiy = reichster Bettler      0        0     0   
4  Ich liebe diese Doppelmoral, Russland greift z...      0        0     0   

   Joy  Sadness  Surprise  
0    0        0         1  
1    0        0         1  
2    0        0         0  
3    1        0         0  
4    0        0         0  


In [ ]:
# Save predictions to CSV
result_df.to_csv('predictions.csv', index=False)

print("Predictions saved to 'predictions.csv'.")


Predictions saved to 'predictions.csv'.


In [ ]:
import pandas as pd

# Step 1: Load the predictions CSV file
df = pd.read_csv('predictions.csv')

# Step 2: Replace 'text' column with an 'id' column
df['id'] = [f"deu_dev_track_c_{str(i+1).zfill(5)}" for i in range(len(df))]

# Step 3: Drop the original 'text' column
df = df.drop(columns=['text'])

# Step 4: Reorder columns so 'id' comes first, followed by the emotions in the correct order
df = df[['id',  'Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']]

# Step 5: Save the updated CSV file for submission
df.to_csv('final_submission.csv', index=False)

print("Submission file 'final_submission.csv' has been created.")


Submission file 'final_submission.csv' has been created.
